In [3]:
def insta_searching(word: str) -> str:
    """인스타그램의 검색 결과 URL을 만드는 함수"""
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [57]:
from selenium import webdriver

import time

driver = webdriver.Chrome('C:/Users/gram/Downloads/chromedriver_win32/chromedriver.exe')

url = insta_searching('제주도맛집')
driver.get(url)
time.sleep(2)

In [59]:
# 인스타 로그인을 해야 게시글을 볼 수 있음

def insta_login(driver):
    with open("./user.txt", "r") as f:
        email = f.readline()
        password = f.readline()
        time.sleep(2)

        try:
            login = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
            login.click()
            time.sleep(2)
        except: # noqa
            pass

        facebook_login = driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF')
        facebook_login.click()
        time.sleep(2)

        input_id = driver.find_element_by_id('email')
        input_id.clear()
        input_id.send_keys(email)
        time.sleep(2)

        input_pw = driver.find_element_by_id('pass')
        input_pw.clear()
        input_pw.send_keys(password)
        input_pw.submit()
        time.sleep(5)
        
        try:
            account = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
            account.click()
            time.sleep(3)
        except:  # noqa
            pass

In [28]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(3)
    
select_first(driver)

In [29]:
from bs4 import BeautifulSoup

import re
import unicodedata

def get_content(driver):
    time.sleep(2)
    
    # 현재 페이지 HRML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except: # noqa
        content = ''

    # 본문에서 해시태그 가져오기
    tags = re.findall(r'#[^/s#,\\]+', content)

    # 작성일자 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]

    # 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except: # noqa
        like = 0

    # 조회 수 가져오기
    try:
        watch = soup.select('div.HbPOm._9Ytll > span')[0].text[3:-1]
    except: # noaq
        watch = 0

    # 위치 정보 가져오기
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except: # noqa
        place = ''

    # 수집한 정보 저장
    data = {'content': content, 'date': date, 'like': like, 'watch': watch, 'place': place}
    return data

get_content(driver)

{'content': '숙성고기의 끝판왕제주육대표 최상급 한우와제주 흑돼지를진공 포장하여 1.5도 물에504시간 동안 숙성시켜서더욱 부드럽고 감칠맛이 풍부한고기를 경험해보세요♥<제주육대표>＊제주시 일주서로 7895＊064-747-0092#제주도맛집#제주공항근처맛집#제주시맛집#제주흑돼지맛집#제주도흑돼지맛집#제주도맛집베스트#제주도맛집투어 #제주도맛집여행#제주도맛집지도#제주맛집찾기',
 'date': '2021-01-01',
 'like': 0,
 'watch': '2,790',
 'place': ''}

In [33]:
def move_to_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
move_to_next(driver)

In [60]:
# 인스타그램 크롤링 전체 과정

from selenium import webdriver
from bs4 import BeautifulSoup

import re
import time
import unicodedata

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Users/gram/Downloads/chromedriver_win32/chromedriver.exe')

# 인스타그램 접속
url = insta_searching('제주도맛집')
driver.get(url)
time.sleep(2)

# 게시글 조회를 위한 인스타그램 로그인
insta_login(driver)

# 인스타그램 검색
url = insta_searching('제주도맛집')
driver.get(url)
time.sleep(2)

# 첫 번째 게시글 조회
select_first(driver)

results = []

target = 200  # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver)
    results.append(data)
    move_to_next(driver)
    
print(results[:2])

[{'content': '숙성고기의 끝판왕제주육대표 최상급 한우와제주 흑돼지를진공 포장하여 1.5도 물에504시간 동안 숙성시켜서더욱 부드럽고 감칠맛이 풍부한고기를 경험해보세요♥<제주육대표>＊제주시 일주서로 7895＊064-747-0092#제주도맛집#제주공항근처맛집#제주시맛집#제주흑돼지맛집#제주도흑돼지맛집#제주도맛집베스트#제주도맛집투어 #제주도맛집여행#제주도맛집지도#제주맛집찾기', 'date': '2021-01-01', 'like': 0, 'watch': '2,810', 'place': ''}, {'content': '비주얼 그대로의 두툼한 흑돼지입니다⠀화산암반수를 먹인무항생제의 제주 흑돼지!품질을 인정받아 백화점에도납품하고 있는 흑돼지입니다🐷🐽⠀흑돼지를 편히 드시도록초벌까지는 저희가 작업을해드리고 있습니다.⠀또한 유아(7세이하) 상차림비는받고 있지 않으니가족여행에서 가족식사로도저렴하게 드실 수 있습니다🥰⠀탐나돈정육점식당▪제주 서귀포시 중문관광로 321▪매일 11:00 - 23:30▪064-738-8700▪010-3640-8833▪단체석,주차#중문맛집 #서귀포맛집#제주맛집 #제주도맛집#제주중문맛집#중문관광단지맛집 #제주도중문맛집 #중문카페 #서귀포카페 #서귀포가볼만한곳', 'date': '2020-12-31', 'like': 0, 'watch': '4,803', 'place': ''}]
